In [1]:
import numpy as np
import pandas as pd
# Setting random seed to get reproducible runs
RSEED = 100

# Importing dataset and processing it

In [2]:
df = pd.read_csv("../data_clean/cancer_industry.csv")

In [3]:
df.head()

,locale,fips,areatype,cancer,stateFIPS,state,cancer_description,annual_count_avg,incidence rate_per_100000,incidence rate_lower_95_confidence,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,"Autauga County(6,10)",1001,county,1,1,alabama,All Cancer Sites,304,495.6,470.6,...,8.749380e-08,0.0,0.004263,0.000000,5.005340e-10,1.617190e-05,0.000365,0.026608,0.869459,0.180875
1,"Autauga County(6,10)",1001,county,1,1,alabama,All Cancer Sites,304,495.6,470.6,...,0.000000e+00,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
2,"Autauga County(6,10)",1001,county,1,1,alabama,All Cancer Sites,304,495.6,470.6,...,0.000000e+00,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,"Autauga County(6,10)",1001,county,1,1,alabama,All Cancer Sites,304,495.6,470.6,...,1.687070e-04,0.0,0.084219,1558.288943,6.755460e-06,2.148530e-08,1050.804066,8.594629,42.953215,26.761900
4,"Autauga County(6,10)",1001,county,1,1,alabama,All Cancer Sites,304,495.6,470.6,...,0.000000e+00,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000


In [4]:
df.columns

Index(['locale', 'fips', 'areatype', 'cancer', 'stateFIPS', 'state',
       'cancer_description', 'annual_count_avg', 'incidence rate_per_100000',
       'incidence rate_lower_95_confidence',
       'incidence rate_upper_95_confidence', 'county', 'name', 'industry_code',
       'industry_detail', 'relevant_naics', 'payann', 'total_compensation',
       'added_value ($)', 'local_tranformation_ind', 'estab', 'emp', 'ACID',
       'ENRG', 'ETOX', 'EUTR', 'FOOD', 'GCC', 'HAPS', 'HAZW', 'HC', 'HNC',
       'HRSP', 'HTOX', 'JOBS', 'LAND', 'METL', 'MINE', 'MSW', 'NREN', 'OZON',
       'PEST', 'REN', 'SMOG', 'VADD', 'WATR'],
      dtype='object')

In [5]:
df.shape

(7030318, 46)

## Only keeping the values that we are interested in

### Keeping general statistics for all types of cancer

In [6]:
dataset = df[df['cancer'] == 1].copy()
dataset.shape

(305666, 46)

### Keeping the columns that we are interested in

In [8]:
# for now we drop the 'fips' column because we supose that the effect of having an idustry present in a certain
# area will be the same in any location
dataset.drop(['locale', 'fips','areatype', 'cancer', 'stateFIPS', 'state',
              'cancer_description', 'annual_count_avg', 'incidence rate_per_100000',
              'incidence rate_lower_95_confidence','incidence rate_upper_95_confidence',
              'industry_detail', 'relevant_naics','county', 'name', 'local_tranformation_ind',
              'total_compensation', 'added_value ($)'], axis=1, inplace=True)

In [9]:
dataset.tail()

,industry_code,payann,estab,emp,ACID,ENRG,ETOX,EUTR,FOOD,GCC,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
7029059,812200,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
7029060,812300,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
7029061,813100,0.177,8,15,0.000001,0.0,0.000001,1.472570e-07,0.000143,0.004671,...,2.589830e-11,0.0,0.001015,0.0,0.0,0.0,0.0,0.000049,0.108083,0.000396
7029062,813a00,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
7029063,813b00,0.000,2,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


# Now we want to predict the 24 environemental factors from varialbles such as fips, pay_ann, total_compensation, added_value, #estab, #emp

## First we have to create one-hot vectors for idustry_codes and fips values

In [10]:
#initializing 1-hot vectors to 0
for ind_code in dataset["industry_code"].unique():
    dataset[ind_code] = 0
    dataset[ind_code] = dataset[ind_code].astype(np.uint8)

In [11]:
dataset.tail()

,industry_code,payann,estab,emp,ACID,ENRG,ETOX,EUTR,FOOD,GCC,...,322291,311230,332913,334300,335221,325413,333991,311221,335110,335224
7029059,812200,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7029060,812300,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7029061,813100,0.177,8,15,0.000001,0.0,0.000001,1.472570e-07,0.000143,0.004671,...,0,0,0,0,0,0,0,0,0,0
7029062,813a00,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7029063,813b00,0.000,2,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [12]:
def set_one_hot(row):
    row[row['industry_code']] = 1
    return row

In [13]:
dataset = dataset.apply(set_one_hot, axis=1)

In [14]:
len(dataset['industry_code'].unique())

335

In [15]:
dataset.tail()

,industry_code,payann,estab,emp,ACID,ENRG,ETOX,EUTR,FOOD,GCC,...,322291,311230,332913,334300,335221,325413,333991,311221,335110,335224
7029059,812200,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7029060,812300,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7029061,813100,0.177,8,15,0.000001,0.0,0.000001,1.472570e-07,0.000143,0.004671,...,0,0,0,0,0,0,0,0,0,0
7029062,813a00,0.000,1,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7029063,813b00,0.000,2,0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#print(list(dataset.columns))

## Partitionning our dataset into train and test sets

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x1 = pd.concat([dataset.loc[:, 'payann':'emp'], dataset.loc[:, '113000':]], axis=1)
y = dataset.loc[:, 'ACID':'WATR']

X_train1, X_test1, y_train, y_test = train_test_split(x1, y,
                                                    test_size=0.3,
                                                    random_state = RSEED)

scaler = StandardScaler()
scaler.fit(X_train1)

X_train = scaler.transform(X_train1)
X_test = scaler.transform(X_test1)

print("XTrain",X_train.shape)
print("XTest",X_test.shape)

XTrain (213966, 338)
XTest (91700, 338)


## Linear Regression

### Scaled X_train and X_test

In [18]:
from sklearn.linear_model import LinearRegression

lr_scaled = LinearRegression()

lr_scaled.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
from sklearn import metrics
train_rf_predictions = lr_scaled.predict(X_train)
rf_predictions = lr_scaled.predict(X_test)
r2_train = metrics.r2_score(y_train, train_rf_predictions)

r2_test = metrics.r2_score(y_test, rf_predictions)

print('r2_score Train using metrics.r2_score:',r2_train)
print('r2_score Train using built in score fct:', lr_scaled.score(X_train, y_train))
print()

print('r2_score Test using metrics.r2_score:', r2_test)
print('r2_score Test using built in score fct:', lr_scaled.score(X_test, y_test))

r2_score Train using metrics.r2_score: 0.11104506552127728
r2_score Train using built in score fct: 0.021568898946116938

r2_score Test using metrics.r2_score: 0.11149949476922068
r2_score Test using built in score fct: 0.0388868300359007


/Users/ghitafilali/anaconda3/envs/cse6242/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/Users/ghitafilali/anaconda3/envs/cse6242/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the buil

In [20]:
from sklearn.model_selection import cross_val_score

cross_val_score(lr_scaled, scaler.transform(x1), y, cv=5, scoring='r2')

array([ 0.11879998,  0.10921225, -0.45351674,  0.11607987,  0.1152629 ])

### Not scaled X_train and X_test

In [21]:
lr_not_scaled = LinearRegression()

lr_not_scaled.fit(X_train1, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
from sklearn import metrics
train_rf_predictions = lr_not_scaled.predict(X_train1)
rf_predictions = lr_not_scaled.predict(X_test1)
MAE_train = metrics.mean_absolute_error(y_train, train_rf_predictions)
RMSE_train = np.sqrt(metrics.mean_squared_error(y_train, train_rf_predictions))
MAPE_train = 100 * np.mean(abs(train_rf_predictions - y_train)/abs(y_train))
accuracy_train = 100 - MAPE_train
r2_train = metrics.r2_score(y_train, train_rf_predictions)

MAE_test = metrics.mean_absolute_error(y_test, rf_predictions)
RMSE_test = np.sqrt(metrics.mean_squared_error(y_test, rf_predictions))
MAPE_test = 100 * np.mean(abs(rf_predictions - y_test)/abs(y_test))
accuracy_test = 100 - MAPE_test
r2_test = metrics.r2_score(y_test, rf_predictions)
#print('Mean Absolute Error Train:', MAE_train)    
#print('Root Mean Squared Error Train:', RMSE_train)
#print('Mean Absolute Percentage Error Train:', MAPE_train)
print('r2_score Train:',r2_train)
print()

print('r2_score Test:', r2_test)

r2_score Train: 0.11104523851276588

r2_score Test: 0.11150001835481553


In [23]:
cross_val_score(lr_not_scaled, x1, y, cv=5, scoring='r2')

array([ 0.11880062,  0.10921096, -0.45353114,  0.11607893,  0.11526358])